### 라이브러리 Import

In [42]:
import os

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from typing import Optional, Any

from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain

from dotenv import load_dotenv
load_dotenv()

True

### 시스템 설정

In [25]:
# Chroma
CHROMA_DB_PATH = "./chroma_db" 

# Neo4j
USERNAME = os.environ.get("USERNAME")
PASSWORD = os.environ.get("PASSWORD")
URL = os.environ.get("URL")

# OpenAI
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)

---

### RAG

In [ ]:
# RAG 프롬프트

RAG_PROMPT_TEMPLATE = """
당신은 Bella Roma 레스토랑 데이터 전문가를 위한 유용한 AI 어시스턴트입니다.
제공된 Context (검색된 문서)만을 사용하여 사용자의 질문에 답변하십시오.
만약 Context에 답변할 수 있는 정보가 없다면, '제공된 정보로는 답변할 수 없습니다.'라고 명확하게 언급하세요.

[질문]
{question}

[Context (검색된 문서)]
{context}
"""

In [ ]:
# ChromaDB 초기화

def initialize_chroma_db(embedding_function, collection_name="unified_data"):
    """ ChromaDB 벡터 스토어를 초기화합니다. (가정: 데이터는 이미 로드되어 있습니다) """
    try:
        # 이미 데이터가 존재하는 컬렉션에 연결합니다.
        vector_db = Chroma(
            persist_directory=CHROMA_DB_PATH,
            embedding_function=embedding_function,
            collection_name=collection_name
        )
        
        collection = getattr(vector_db, "_collection", None)
        count = collection.count()
        print(f"연결된 컬렉션(unified_data) 문서 개수: {count}")
        print()
        
        return vector_db
    except Exception as e:
        print(f"ChromaDB 초기화 오류: {e}")
        return None

In [ ]:
# RAG 파이프라인

def rag_pipeline(question: str, llm: ChatOpenAI) -> str:
    """
    Vector DB (ChromaDB)를 사용하여 비정형 데이터 기반 RAG 파이프라인을 LCEL로 실행합니다.
    """
    # 1. Vector DB 초기화 및 임베딩 함수 설정
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vector_db = initialize_chroma_db(embeddings, collection_name="unified_data")
    
    if vector_db is None:
        return "ERROR: Vector DB를 초기화할 수 없습니다."

    # 2. Retriever 및 Prompt Template 정의
    retriever = vector_db.as_retriever(search_kwargs={"k": 5})
    prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

    # 3. LCEL 체인 구성 (RunnablePassthrough 사용)
    # 체인 흐름: 
    # {context: 검색(retriever), question: 질문 원문(RunnablePassthrough)} 
    # -> Prompt -> LLM -> 결과 파싱
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    # 4. 질문 실행 및 결과 생성
    try:
        # invoke 메서드는 LCEL 체인을 실행하며, 입력은 question 문자열입니다.
        result = rag_chain.invoke(question)
        return result
    except Exception as e:
        return f"ERROR: RAG 파이프라인 실행 오류: {e}"

# 예시 사용법 (실제 실행 시 주석 해제)
question = "Bella Roma의 주중 운영 시간은 무엇이며, 어떤 결제 수단을 받나요?"
# question = "Bella Roma 레스토랑의 영업시간을 알려주세요."
# llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0) # llm 객체 초기화 필요
answer = rag_pipeline(question, llm)
print(f"RAG 답변: {answer}")

연결된 컬렉션(unified_data) 문서 개수: 608
RAG 답변: 주중(월~금) 영업시간은 11:00부터 22:00까지입니다.  
결제 수단은 현금, 카드 및 다양한 간편결제를 지원합니다.


### Graph RAG

In [5]:
def initialize_neo4j_graph() -> Neo4jGraph:
    """ Neo4jGraph 객체를 초기화합니다. """
    # 실제 환경에서는 NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD를 사용해야 합니다.
    try:
        # 임시 URL/인증 정보를 가정하거나 환경 변수에서 로드합니다.
        graph = Neo4jGraph(
            url="bolt://52.3.233.24:7687", # 실제 URL로 변경 필요
            username="neo4j", 
            password="canvases-return-armaments"
        )
        # 스키마가 미리 로드되어 있다고 가정
        return graph
    except Exception as e:
        print(f"Neo4j 초기화 오류: {e}")
        return None

In [10]:
def graph_rag_pipeline(question: str, llm: ChatOpenAI) -> str:
    """
    Graph DB (Neo4j)를 사용하여 스키마 기반 Cypher 쿼리 생성 파이프라인을 실행합니다.
    """
    # 1. Neo4j Graph 초기화
    graph = initialize_neo4j_graph()
    if graph is None:
        return "ERROR: Neo4j Graph를 초기화할 수 없습니다."
    
    # 2. GraphCypherQAChain 생성
    # LLM에 스키마를 전달하여 Cypher 쿼리 생성 및 실행 후 답변 생성
    qa_chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        verbose=False,
        allow_dangerous_requests=True,
        # LLM이 쿼리를 생성하고 결과를 자연어로 해석할 때 스키마가 Context로 사용됨
    )
    
    # 3. 질문 실행 및 결과 생성
    try:
        result = qa_chain.invoke({"query": question})
        return result["result"]
    except Exception as e:
        return f"ERROR: GRAPH RAG 파이프라인 실행 오류: {e}"

# # 예시 사용법 (실제 실행 시 주석 해제)
# question = "마르게리타 피자는 어떤 재료를 포함하며, 유제품 알러지를 유발하나요?"
# answer = graph_rag_pipeline(question, llm)
# print(f"GRAPH RAG 답변: {answer}")

### Hybrid (RAG & Graph RAG)

In [ ]:
# 하이브리드 RAG 프롬프트

HYBRID_QA_TEMPLATE = """
당신은 Bella Roma 레스토랑 데이터 분석 전문가입니다.
아래에 제공된 두 가지 검색 결과 (정형 데이터 및 비정형 데이터)를 모두 활용하여 사용자의 질문에 대한 가장 정확하고 포괄적인 답변을 한국어로 생성하십시오.

[질문]
{question}

--- Graph DB (정형 데이터) Context ---
{graph_context}
---------------------------------------

--- Vector DB (비정형 데이터) Context ---
{vector_context}
-----------------------------------------

# 응답 가이드라인:
- 두 Context를 종합하여 답변하며, 충돌하는 정보가 있을 경우 정형 데이터(Graph DB)의 사실 정보를 우선하세요.
- 답변은 전문적이고 유익한 톤을 사용하세요.
- 두 Context 모두에 답변할 정보가 없는 경우, '제공된 정보로는 답변을 통합할 수 없습니다.'라고 명확히 언급하세요.
"""

In [36]:
HYBRID_QA_PROMPT = ChatPromptTemplate.from_template(HYBRID_QA_TEMPLATE)

In [ ]:
def initialize_chroma_db(embedding_function: OpenAIEmbeddings, collection_name: str) -> Optional[Chroma]:
    """ ChromaDB 벡터 스토어를 초기화합니다. """
    try:
        vector_db = Chroma(
            persist_directory=CHROMA_DB_PATH,
            embedding_function=embedding_function,
            collection_name=collection_name
        )
        # print(f"연결된 컬렉션({collection_name}) 문서 개수: {getattr(vector_db, '_collection').count()}")
        return vector_db
    except Exception as e:
        print(f"ChromaDB 초기화 오류: {e}")
        return None

In [ ]:
def initialize_neo4j_graph() -> Optional[Neo4jGraph]:
    """ Neo4jGraph 객체를 초기화합니다. """
    try:
        graph = Neo4jGraph(url=URL, username=USERNAME, password=PASSWORD)
        return graph
    except Exception as e:
        print(f"Neo4j 초기화 오류: {e}")
        return None

In [39]:
def fetch_vector_context(question: str, llm: ChatOpenAI) -> str:
    """ Vector DB에서 관련 문서의 Context를 검색합니다. """
    try:
        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        vector_db = initialize_chroma_db(embeddings, collection_name="unified_data")
        
        if vector_db is None:
            return "ERROR: Vector DB 초기화 실패"
        
        # 5개 문서를 검색하여 텍스트 Context로 변환
        retriever = vector_db.as_retriever(search_kwargs={"k": 5})
        docs = retriever.invoke(question)
        
        # 검색된 문서 내용을 하나의 문자열로 결합
        context = "\n---\n".join([doc.page_content for doc in docs])
        return context
        
    except Exception as e:
        return f"ERROR: Vector DB 검색 오류: {e}"

In [41]:
def fetch_graph_context(question: str, llm: ChatOpenAI) -> str:
    """ Graph DB에서 Cypher 쿼리를 생성 및 실행하여 결과를 반환합니다. """
    graph = initialize_neo4j_graph()
    if graph is None:
        return "ERROR: Neo4j Graph 초기화 실패"
    
    # GraphCypherQAChain 생성 및 실행
    qa_chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        verbose=False,
        allow_dangerous_requests=True,
    )
    
    try:
        # LLM은 Cypher 쿼리 실행 결과를 바탕으로 자연어 Context를 생성하여 반환합니다.
        result = qa_chain.invoke({"query": question})
        
        # GraphCypherQAChain의 결과는 이미 자연어 답변 형태이지만,
        # 여기서는 LLM이 최종 답변을 만들기 위한 'Context'로 사용되므로,
        # Cypher 쿼리와 결과를 모두 포함하는 디버그 정보로 전달하는 것이 이상적입니다.
        # (편의상 여기서는 chain의 최종 result만 사용합니다.)
        return result["result"]
    
    except Exception as e:
        return f"ERROR: GRAPH RAG 실행 오류: {e}"

In [ ]:
# 하이브리드 파이프라인

def hybrid_rag_pipeline(question: str, llm: ChatOpenAI) -> str:
    """
    Graph DB (정형)와 Vector DB (비정형)의 Context를 결합하여 최종 답변을 생성합니다.
    """
    print("--- 1. Graph DB 컨텍스트 검색 시작 ---")
    # (a) Graph DB 검색 - 정형 데이터 및 관계 정보
    graph_context = fetch_graph_context(question, llm)
    print(f"Graph Context: {graph_context[:100]}...")

    print("--- 2. Vector DB 컨텍스트 검색 시작 ---")
    # (b) Vector DB 검색 - 비정형 텍스트 문서
    vector_context = fetch_vector_context(question, llm)
    print(f"Vector Context: {vector_context[:100]}...")

    # 3. Context 결합 및 최종 답변 생성 (LCEL 사용)
    
    # Context를 결합하는 Runnable
    combined_context_runnable = RunnableLambda(
        lambda q: {
            "question": q, 
            "graph_context": graph_context, 
            "vector_context": vector_context
        }
    )
    
    # LCEL 체인 구성: 입력 -> Context 결합 -> Prompt -> LLM -> 결과 파싱
    hybrid_chain = (
        combined_context_runnable
        | HYBRID_QA_PROMPT
        | llm
        | StrOutputParser()
    )
    
    print("--- 3. 하이브리드 LLM 추론 시작 ---")
    try:
        final_answer = hybrid_chain.invoke(question)
        return final_answer
    except Exception as e:
        return f"ERROR: 하이브리드 LLM 추론 오류: {e}"

In [ ]:
# 실행 예시 (사용자가 직접 실행) 

question_hybrid = "2024년 8월에 가장 많이 팔린 메뉴는 무엇인가요?"
llm_instance = ChatOpenAI(model_name="gpt-4o-mini", temperature=0) 
answer = hybrid_rag_pipeline(question_hybrid, llm_instance)
print("\n=== 최종 하이브리드 RAG 답변 ===")
print(answer)

--- 1. Graph DB 컨텍스트 검색 시작 ---
Graph Context: 2024년 8월에 가장 많이 팔린 메뉴는 까르보나라 파스타로, 총 35개가 판매되었습니다....
--- 2. Vector DB 컨텍스트 검색 시작 ---
Vector Context: 사용자 사용자8이 2024-10-03에 평점 3점으로 '분위기가 좋아서 데이트 장소로 딱입니다.'라는 리뷰를 남겼습니다.
---
사용자 사용자4이 2024-10-06에 평점 4점으...
--- 3. 하이브리드 LLM 추론 시작 ---

=== 최종 하이브리드 RAG 답변 ===
2024년 8월에 가장 많이 팔린 메뉴는 까르보나라 파스타로, 총 35개가 판매되었습니다. 이 메뉴는 고객들 사이에서 인기가 높아 데이트 장소로도 적합하다는 긍정적인 리뷰가 여러 건 남겨졌습니다. 예를 들어, 사용자8과 사용자4는 각각 '분위기가 좋아서 데이트 장소로 딱입니다.'라는 리뷰를 남겼습니다. 이러한 리뷰는 까르보나라 파스타가 맛뿐만 아니라 레스토랑의 분위기와도 잘 어울린다는 점을 강조합니다. 

따라서, 2024년 8월의 판매 데이터와 고객 리뷰를 종합적으로 고려할 때, 까르보나라 파스타는 고객들에게 매우 긍정적인 반응을 얻고 있는 메뉴임을 알 수 있습니다.
